<a href="https://colab.research.google.com/github/mopfun/Project_4_Grp_1/blob/MI/Netflix_Top_10_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing Data

In [10]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import pandas as pd


In [11]:
import os
os.getcwd()

'/content'

# Make sure you upload final_combined_data.csv to the **content folder** in google colab every time you run this file

In [21]:
# Import in csv file
df = pd.read_csv("final_combined_data.csv")
df.head()

,row_id,movie_name,rating,votes,meta_score,genre0,genre1,genre2,genre3,pr_rating,...,cast1,cast2,cast3,cast4,director,number_rating,weekly_rank,weekly_hours_viewed,weekly_views,cumulative_weeks_in_top_10
0,506,1917,8.2,654000.0,78,"Action, Drama, War",Action,Drama,War,R,...,Dean-Charles Chapman,George MacKay,Daniel Mays,Colin Firth,Sam Mendes,4,7,10840000,NaN,1
1,1356,21 Jump Street,7.2,589000.0,69,"Action, Comedy, Crime",Action,Comedy,Crime,R,...,Jonah Hill,Channing Tatum,Ice Cube,Brie Larson,Phil LordChristopher Miller,4,6,11690000,NaN,1
2,1698,6 Underground,6.1,192000.0,41,"Action, Thriller",Action,Thriller,NaN,R,...,Ryan Reynolds,Mélanie Laurent,Manuel Garcia-Rulfo,Ben Hardy,Michael Bay,4,5,8730000,NaN,3
3,1698,6 Underground,6.1,192000.0,41,"Action, Thriller",Action,Thriller,NaN,R,...,Ryan Reynolds,Mélanie Laurent,Manuel Garcia-Rulfo,Ben Hardy,Michael Bay,4,7,7440000,NaN,2
4,1698,6 Underground,6.1,192000.0,41,"Action, Thriller",Action,Thriller,NaN,R,...,Ryan Reynolds,Mélanie Laurent,Manuel Garcia-Rulfo,Ben Hardy,Michael Bay,4,9,7770000,NaN,1


In [22]:
# Drop columns?
df = df.drop(columns=["row_id", "movie_name", "genre0", "cast0", "number_rating", "weekly_hours_viewed"])

In [23]:
# Determine the number of unique values in each column.
df.nunique()

,0
rating,36
votes,155
meta_score,63
genre1,8
genre2,16
genre3,15
pr_rating,5
year,30
duration,66
cast1,127


In [25]:
# Look at specific value counts to identify and replace with "Other"
df['director'].value_counts()

,count
director,
Brad Furman,19
Garth JenningsChristophe Lourdelet,18
Rawson Marshall Thurber,16
Greta Gerwig,11
Gillian Armstrong,11
...,...
Roar Uthaug,1
J.C. Chandor,1
Adrian Lyne,1


In [ ]:
# # Choose a cutoff value and create a list of application types to be replaced
# application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# # Replace in dataframe
# for app in application_types_to_replace:
#     application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure replacement was successful
# application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`(won't change numerical columns, only categorical)
categorical_dummies = pd.get_dummies(df)

In [ ]:
## Reuse this data for all the models
# Split our preprocessed data into our features and target arrays
X = df.drop("cumulative_weeks_in_top_10", axis=1)
y = df["cumulative_weeks_in_top_10"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
classification_dummies.shape[1]

# Linear Regression Model

We started with a **Linear Regression model** as a simple and interpretable baseline. This helped us:

- Quickly test if a linear relationship exists between the features and the target
- Understand which features have the strongest impact on the outcome
- Provide an easy-to-explain model using coefficients
- Compare performance against more complex models like Random Forest and Keras

Even if it's not the best performer, it gives us a valuable starting point for analysis and model comparison.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Create and fit the model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predict
predicitions = lr_model.predict(X_test_scaled)


In [ ]:
# Evaluate
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print(f"Linear Regression Mean Absolute Error (mae): {mae:.2f}")
print(f"Linear Regression Root Mean Squared Error: {rmse:.2f}")
print(f"Linear Regression R2: {r2:.2f}")

# Save Model
lr_model.save('linear_regression_model.keras')

### What These Metrics Tell Us

- **MAE (Mean Absolute Error)**: This tells us how far off our predictions are on average, measured in weeks.
- **RMSE (Root Mean Squared Error)**: Similar to MAE, but it puts more weight on larger mistakes.
- **R2 (R-squared)**: Shows how much of the variation in the target the model explains. A score closer to 1 means the model fits the data well.


# Random Forest Model

We used a **Random Forest Regressor** to improve predictive performance over the linear model. This method:

- Handles complex, nonlinear relationships without requiring feature engineering
- Is robust to outliers and overfitting due to its use of **multiple decision trees**
- Automatically detects feature importance and interactions
- Requires minimal preprocessing and works well with both numeric and encoded categorical data

It’s a strong, reliable model for tabular data and a valuable benchmark before moving to deep learning.

In [ ]:
# Import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

## Random forest DOES NOT use scaled data
# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)


In [ ]:
# Evaluate the model
mae_rf = mean_absolute_error(y_test, rf_predictions)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_predictions))
r2_rf = r2_score(y_test, rf_predictions)

print(f"Random Forest Mean Absolute Error: {mae_rf:.2f}")
print(f"Random Forest Root Mean Squared Error: {rmse_rf:.2f}")
print(f"Random Forest R2 Score: {r2_rf:.2f}")

#Save model
rf_model.save('random_forest_model.keras')

# Keras Neural Network Model

We used a **Keras neural network** to explore whether deep learning can capture more complex patterns in the data. This model:

- Learns nonlinear relationships and deep feature interactions
- Can potentially outperform traditional models with enough data and tuning
- Requires feature scaling and hyperparameter tuning for best results
- Is highly customizable in terms of architecture and optimization

This model helps us test if complexity leads to better predictive performance, especially after comparing it with simpler methods.


In [ ]:
# Create the Keras Model

# This counts how many features (columns) your data has, so the model knows what kind of input to expect.
number_input_features = len(X_train[0])

# Used the standard 2 layers and standard neurons(64 and 32) to run a quick test
hidden_nodes_layer1 = 64
hidden_nodes_layer2 = 32

# Start building the Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim= number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer2, activation="relu"))

# Output layer – Removed sigmoid because we are predicting a continuous number (weeks), not a binary value (0 or 1)
nn.add(tf.keras.layers.Dense(units=1))

# Check the structure of the model
nn.summary()


In [ ]:
# Compile the model
# loss='mse' directs the model to minimize mean squared error during training
# metrics=['mae'] tracks the average prediction error (in weeks) for easier interpretation
nn.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Train the model
# validation_split=0.2 reserves 20% of training data for validation, helping us monitor overfitting and track performance on unseen data.
# batch_size=32 trains the model in small groups, balancing speed, memory use, and generalization.
fit_model_history = nn.fit(X_train_scaled, y_train, validation_split=0.2, epochs=50, batch_size=32)


In [ ]:
# Evaluate the model on test data
# MAE (Mean Absolute Error) shows how many weeks off the predictions are, on average
loss, mae = nn.evaluate(X_test_scaled, y_test)
print(f"Test MAE: {mae:.2f}, Loss (MSE): {loss:.2f}")

#Save file
nn.save('keras_model.keras')

# Keras Tuner

We used **Keras Tuner** to optimize our neural network’s architecture and hyperparameters. While our base Keras model gives us a starting point, Keras Tuner helps us:

- Automatically search for the best combination of layers, units, activations, and learning rates
- Improve model performance without manually guessing parameters
- Reduce overfitting or underfitting by finding better model configurations
- Explore multiple architectures efficiently

This step helps us fine-tune our model and potentially outperform traditional machine learning methods by leveraging the flexibility of deep learning.


In [ ]:
# This counts how many features (columns) your data has, so the model knows what kind of input to expect.
number_input_features = len(X_train[0])

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','selu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=8,
        max_value=128,
        step=8), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=8,
            max_value=64,
            step=8),
            activation=activation))

    # Output layer for regression (no activation)
    nn.add(tf.keras.layers.Dense(units=1))

    # Compile the model
    nn.compile(loss="mse", optimizer="adam", metrics=["mae"])

    return nn

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_mae",         # Mean Absolute Error for regression
    max_epochs=20,
    hyperband_iterations=2)


In [ ]:
# Run the Keras Tuner search for best hyperparameters
tuner.search(X_train_scaled, y_train,
             epochs=20,
             validation_split=0.2,
             batch_size=32)


In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]

# Evaluate it on the test set
loss, mae = best_model.evaluate(X_test_scaled, y_test)
print(f"Best Tuned Model: Test Mean Absolute Error: {mae:.2f}")

# Save the model to a file
best_model.save('keras_tuner_best_model.keras')